In [1]:
import os 
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

In [2]:
import torch
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# Own library imports
from utilities.utils import TensOps
from utilities.operators.zero_order import Mx, My
from utilities.kernels.derivative import DerivativeKernels

# Function from this project
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop, train_autoencoder_loop

# Import model
from architectures.autoencoder import Autoencoder
from architectures.pgnniv_decoder import PGNNIVAutoencoder

In [3]:
# Dataset
dataset = 'non_linear'
N_data = 100
noise = 1

data_name = dataset + '_' + str(N_data) + '_' + str(noise)

In [4]:
# Model
model = 'autoencoder'
n_modes = 10

model_name = model + '_model_' + str(n_modes)

In [5]:
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../"))
DATA_PATH = os.path.join(ROOT_PATH, r'data/', data_name, data_name) + '.pkl'
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/', data_name)

MODEL_RESULTS_AE_PATH = os.path.join(ROOT_PATH, r'results/', data_name, model_name) + '_AE'
MODEL_RESULTS_PGNNIV_PATH = os.path.join(ROOT_PATH, r'results/', data_name, model_name) + '_NN'

# Create folders (if necessary)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_AE_PATH)
create_folder(MODEL_RESULTS_PGNNIV_PATH)

Folder successfully created at: c:\Users\usuario\Desktop\rmunozTMELab\Embedding-Oriented-PGNNIV\results/non_linear_100_1
Folder successfully created at: c:\Users\usuario\Desktop\rmunozTMELab\Embedding-Oriented-PGNNIV\results/non_linear_100_1\autoencoder_model_10_AE
Folder successfully created at: c:\Users\usuario\Desktop\rmunozTMELab\Embedding-Oriented-PGNNIV\results/non_linear_100_1\autoencoder_model_10_NN


In [6]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: c:\Users\usuario\Desktop\rmunozTMELab\Embedding-Oriented-PGNNIV\data/non_linear_100_1\non_linear_100_1.pkl


In [7]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [8]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cpu


In [9]:
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
y_train = torch.Tensor(dataset['y_train']).unsqueeze(1)
K_train = torch.tensor(dataset['k_train']).unsqueeze(1)
f_train = torch.tensor(dataset['f_train']).unsqueeze(1).to(torch.float32)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1)
y_val = TensOps(torch.Tensor(dataset['y_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val']).to(torch.float32).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

print("Train dataset length:", len(X_train))
print("Validation dataset length:", len(X_val))

Train dataset length: 80
Validation dataset length: 20


In [10]:
N_data_AE = len(X_train)//2
N_data_NN = len(X_train) - len(X_train)//2
prop_data_NN = 1 - N_data_AE/(N_data_NN + N_data_AE)

print("Dataset length for the autoencoder:", N_data_AE)
print("Dataset length for the PGNNIV:", N_data_NN)

X_AE, X_NN, y_AE, y_NN, K_AE, K_NN, f_AE, f_NN = train_test_split(X_train, y_train, K_train, f_train, test_size=prop_data_NN, random_state=42)

Dataset length for the autoencoder: 40
Dataset length for the PGNNIV: 40


In [11]:
y_train_AE, y_test_AE = train_test_split(y_AE, test_size=0.2, random_state=42)

y_train_AE = TensOps(y_train_AE.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
y_test_AE = TensOps(y_test_AE.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

In [12]:
X_train_NN, X_test_NN, y_train_NN, y_test_NN, K_train_NN, K_test_NN, f_train_NN, f_test_NN = train_test_split(X_NN, y_NN, K_NN, f_NN, test_size=0.2, random_state=42)

X_train_NN = X_train_NN.to(DEVICE)
X_test_NN = X_test_NN.to(DEVICE)

y_train_NN = TensOps(y_train_NN.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
y_test_NN = TensOps(y_test_NN.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

K_train_NN = TensOps(K_train_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
K_test_NN = TensOps(K_test_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

f_train_NN = TensOps(f_train_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
f_test_NN = TensOps(f_test_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

## Autoencoder

In [13]:
autoencoder_input_shape = y_train_AE.values[0].shape
latent_space_dim = [15, 10, n_modes, 10, 15]
autoencoder_output_shape = y_train_AE.values[0].shape

In [14]:
X_train = y_train_AE.values
y_train = y_train_AE

X_test = y_test_AE.values
y_test = y_test_AE

In [15]:
autoencoder = Autoencoder(autoencoder_input_shape, latent_space_dim, autoencoder_output_shape).to(DEVICE)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-2)

start_epoch = 0
n_epochs = 1000
batch_size = 64
n_checkpoint = 10
new_lr = None

train_autoencoder_loop(autoencoder, optimizer, X_train, y_train, X_test, y_test,  
                       n_checkpoint, start_epoch, n_epochs, batch_size, MODEL_RESULTS_AE_PATH, DEVICE, new_lr)

Starting training from scratch.
Epoch 0, Train loss: 5.377e+01, Test loss: 8.336e+01
Epoch 10, Train loss: 6.826e+00, Test loss: 1.071e+01
Epoch 20, Train loss: 3.159e+00, Test loss: 6.532e+00
Epoch 30, Train loss: 2.460e+00, Test loss: 4.526e+00
Epoch 40, Train loss: 2.054e+00, Test loss: 4.155e+00
Epoch 50, Train loss: 1.917e+00, Test loss: 3.747e+00
Epoch 60, Train loss: 1.885e+00, Test loss: 3.685e+00
Epoch 70, Train loss: 1.869e+00, Test loss: 3.683e+00
Epoch 80, Train loss: 1.864e+00, Test loss: 3.660e+00
Epoch 90, Train loss: 1.862e+00, Test loss: 3.679e+00
Epoch 100, Train loss: 1.861e+00, Test loss: 3.660e+00
Epoch 110, Train loss: 1.861e+00, Test loss: 3.667e+00
Epoch 120, Train loss: 1.860e+00, Test loss: 3.664e+00
Epoch 130, Train loss: 1.860e+00, Test loss: 3.665e+00
Epoch 140, Train loss: 1.858e+00, Test loss: 3.665e+00
Epoch 150, Train loss: 1.849e+00, Test loss: 3.664e+00
Epoch 160, Train loss: 1.812e+00, Test loss: 3.658e+00
Epoch 170, Train loss: 1.618e+00, Test loss:

In [ ]:
start_epoch = 900
n_epochs = 2000
batch_size = 64
n_checkpoint = 10
new_lr = 1e-4
4
train_autoencoder_loop(autoencoder, optimizer, X_train, y_train, X_test, y_test,  
                       n_checkpoint, start_epoch, n_epochs, batch_size, MODEL_RESULTS_AE_PATH, DEVICE, new_lr)

Starting training from a checkpoint. Epoch 900.
Epoch 900, Train loss: 2.285e-01, Test loss: 5.855e-01
Epoch 1000, Train loss: 2.280e-01, Test loss: 5.850e-01
Epoch 1100, Train loss: 2.280e-01, Test loss: 5.849e-01


## PGNNIV

In [ ]:
# Predictive network architecture
input_shape = X_train_NN[0].shape
predictive_layers = [15, 10, n_modes]
predictive_output = y_train_NN.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train_NN)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train_NN)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [ ]:
pretrained_decoder = autoencoder.decoder

for param in pretrained_decoder.parameters():
    param.requires_grad = False

for name, param in pretrained_decoder.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

In [ ]:
model = PGNNIVAutoencoder(input_shape, predictive_layers, pretrained_decoder, predictive_output, explanatory_input,
                                   explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Training parameters
start_epoch = 0
n_epochs = 1000

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train_NN, y_train_NN, f_train_NN, X_test_NN, y_test_NN, f_test_NN,
           D, n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PGNNIV_PATH, device=DEVICE)

In [ ]:
# Training parameters
start_epoch = 900
n_epochs = 2000

batch_size = 64 
n_checkpoints = 100

second_lr = 1e-4

train_loop(model, optimizer, X_train_NN, y_train_NN, f_train_NN, X_test_NN, y_test_NN, f_test_NN,
           D, n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PGNNIV_PATH, device=DEVICE, new_lr=second_lr)